In [1]:
import warnings
warnings.simplefilter('ignore')

# Table of Contents
>## 1. What is `Tensorflow`?

>## 2. Operations - Basic
* 2.1. Define Symbols
* 2.2. Define Symbolic Relationship
* 2.3. Define & Run Sessions

>## 3. Operations - Intermediate
* 3.1. Optimization
* 3.2. Differentiation

>## 4. Operations - Advanced

>## 5. TensorBoard
* 5.1. Generating Log File
* 5.2. Deleting Existing Log File
* 5.3. TensorBoard

>## 6. Examples
* 6.1. Perceptron - Iris Dataset

# 1. What is `Tensorflow`?
* `TensorFlow`: Symbolic Linear Algebra Compiler
* Better than `Theano` in distributed processing

# 2. Operations - Basic
* **Steps**:
  * Define Symbols
  * Define Symbolic Relationship
  * Define Session (similar to function in `theano`)
  * Run Session

## 2.1. Define Symbols
* **`Variable` command**
* **Datatypes**: selected based on initial value
* **Name**: not necessary

In [2]:
import tensorflow as tf

In [3]:
x = tf.Variable(1.0)
y = tf.Variable(2.0)

In [4]:
type(x), type(y)

(tensorflow.python.ops.variables.Variable,
 tensorflow.python.ops.variables.Variable)

## 2.2. Define Symbolic Relationship

In [5]:
z = x + y

In [6]:
type(z)

tensorflow.python.framework.ops.Tensor

In [7]:
u = tf.log(z)

In [8]:
type(u)

tensorflow.python.framework.ops.Tensor

## 2.3. Define & Run Sessions
* **1. Create Session**: Generate `Session` object
* **2. `global_variables_initializer`**
* **3. Run Session**: `run` command / calculate & return
* **4. Close Session**: `close` command / often replaced with `with`

In [9]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
print(sess.run(z)) 
print(sess.run(u)) 
sess.close()

3.0
1.0986123


# 3. Operations - Intermediate

## 3.1. Optimization
* Users don't have to optimize the sessions
* Classes like `GradientDescentOptimizer` are already implemented

## 3.2. Differentiation
* `gradients` command

In [10]:
f = x ** 2
fx = tf.gradients(f, [x])

with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    print(sess.run(f))    
    print(sess.run(fx))

1.0
[2.0]


# 4. Operations - Advanced

# 5. TensorBoard

## 5.1. Generating Log File
* **1. `summary.scalar` 명령이나 `summary.histogram` 명령으로 개별 변수 기록용 summary 생성**
* **2. `summary.merge_all` 명령으로 개별 변수 기록용 summary를 하나의 summary로 합침**
* **3. 세션 생성후 `tf.summary.FileWriter` 명령으로 기록용 writer 객체 생성(위 예제에서는 `writer`). 이 때 기록할 디렉토리 설정**
* **4. `sess.run` 명령으로 summary 객체(위 예제에서는 `merged`)를 보내면 기록 실시**
* **5. 기록용 write 객체의 `add_summary` 명령으로 summary를 실제 파일에 쓰기.**

## 5.2. Deleting Existing Log
* Don't forget to delete the log in the directory before running the code

In [11]:
!rm -rf /home/dockeruser/logs/*

## 5.3. TensorBoard
* TensorBoard uses port 6006
* Step 1. Open port

```
docker run --name rpython -it -p 8888:8888 -p 6006:6006 datascienceschool/rpython
```

* Step 2. In terminal,

```
$ tensorboard --host=0.0.0.0 --logdir=/home/dockeruser/logs
```

* Step 3. Go to Tensorboard
  * http://localhost:6006
  * http://192.168.99.100:6006  (If using Docker Toolbox)



<img src="https://datascienceschool.net/upfiles/3763526231d843b5a01775a8cae316a4.png" style="width:100%;">

# 6. Examples

## 6.1. Perceptron - Iris Dataset

In [12]:
from sklearn.datasets import load_iris
iris = load_iris()

idx = np.in1d(iris.target, [0, 2])
X_data = iris.data[idx, 0:2]
y_data = (iris.target[idx] - 1.0)[:, np.newaxis] # y=1 or y=-1

In [13]:
np.random.seed(0)
w = tf.Variable(1e-3 * np.random.randn(2, 1))
b = tf.Variable(1e-3 * np.random.randn(1)[0])
z = tf.tanh(tf.matmul(X_data, w) + b)

zero = np.zeros(1, dtype=np.float64)[0] 
cost = tf.reduce_sum(tf.maximum(zero, tf.multiply(-y_data, z)))
optimizer = tf.train.GradientDescentOptimizer(1e-7)
train = optimizer.minimize(cost)

sess = tf.Session()

w_summary = tf.summary.histogram("w", w)
b_summary = tf.summary.scalar("b", b)
z_summary = tf.summary.histogram("z", z)
cost_summary = tf.summary.scalar("cost", cost)
merged = tf.summary.merge_all()

writer = tf.summary.FileWriter("/home/dockeruser/logs", sess.graph)

init = tf.global_variables_initializer()
sess.run(init)

for i in range(100):
    sess.run(train)
    summary = sess.run(merged)
    writer.add_summary(summary, i)
    if i % 10 == 0:
        print("{:4d}: ".format(i), sess.run(cost))

   0:  0.5494091733308988
  10:  0.4550718147888182
  20:  0.3607278431087564
  30:  0.2663786243415365
  40:  0.17202552496054824
  50:  0.07766991175203047
  60:  0.0009515019028440592
  70:  0.0003168280919044534
  80:  0.0002921058755255818
  90:  0.00029228987473695566


In [14]:
y_pred = sess.run(tf.sign(z))

In [15]:
sess.close()

In [16]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_data, y_pred)

array([[49,  1],
       [ 1, 49]])